In [2]:
!pip install tf_keras scikit-learn wordcloud tensorflow
# !pip install --upgrade ktrain
!pip install Keras-Preprocessing
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.corpus import reuters
from nltk.corpus import brown
from nltk.corpus import gutenberg
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pickle
import joblib
from collections import Counter
from textblob import Word
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, f1_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Activation, Dense, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten, GRU, Conv1D, MaxPooling1D, Bidirectional
from wordcloud import WordCloud,ImageColorGenerator
from PIL import Image
import urllib
import requests
import re
sns.set()
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('gutenberg')
nltk.download('brown')
nltk.download("reuters")
nltk.download('words')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [4]:
df=pd.read_csv("../Data/processedData.csv")
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,no,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,yes,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,yes,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,no,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,no,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,yes,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [5]:
num_species = df['species_group'].nunique()
num_species
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,no,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,yes,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,yes,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,yes,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,no,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,no,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,no,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,yes,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,yes,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [6]:
df['tail']=df['tail'].map({'yes':1, 'no':0})
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_message = vectorizer.fit_transform(df['processedMessage'])

In [8]:
from scipy.sparse import hstack

X_features = hstack([X_message, np.array(df[['fingers', 'tail']])])
y = df['species']

In [9]:
df

,Unnamed: 0,message,fingers,tail,species,preprocessed_message,processedMessage,species_group
0,0,pluvia arbor aquos,4,0,Aquari,"['rain', 'a', 'tree', 'water']",rain tree water,Aquari
1,1,cosmix xeno nebuz odbitaz,5,1,Zorblax,"['cosmos', 'xeno', 'in', 'the', 'sky', 'reject...",cosmos xeno sky rejected,Zorblax
2,2,solarix glixx novum galaxum quasar,5,1,Zorblax,"['solar', 'glix', 'new', 'galaxy', 'quasar']",solar glix new galaxy quasar,Zorblax
3,3,arbor insectus pesros ekos dootix nimbus,2,1,Florian,"['a', 'tree', 'insect', 'peso', 'eco', 'gifted...",tree insect peso eco gifted cloud,Florian
4,4,mermax drakos lorix epikoz deftax,4,0,Faerix,"['merma', 'dragon', 'to', 'wear', 'epic', 'deft']",merma dragon wear epic deft,Faerix_Group2
...,...,...,...,...,...,...,...,...
495,495,empathix sadix disgux dredax pridius afgstix e...,2,0,Emotivor,"['empathize', 'sad', 'discus', 'dreds', 'earli...",empathize sad discus dreds earlier afgst excited,Emotivor_Group2
496,496,quasar ustron nebulax meteorn,4,0,Quixnar,"['quasar', 'burn', 'fog', 'meteorn']",quasar burn fog meteorn,Quixnar
497,497,astron xeno ceaestar astron kometa,6,1,Zorblax,"['astron', 'xeno', 'dinner', 'astron', 'comet']",astron xeno dinner astron comet,Zorblax
498,498,sporzom nimbus terram terranix aviana ekos nimbub,2,1,Florian,"['sporzom', 'cloud', 'the', 'earth', 'earthlin...",sporzom cloud earth earthling avian eco nimbub,Florian


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['species_number'] = label_encoder.fit_transform(df['species_group'])

X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.15, random_state=37)

## Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Function to combine labels
def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })

# Define the Random Forest model with specified parameters
rfc = RandomForestClassifier(n_estimators=300, max_depth=15, random_state=37, class_weight='balanced')

# Train the model
rfc.fit(X_train, y_train)

# Predict using the trained model
y_pred = rfc.predict(X_val)

# Combine the labels for both predicted and actual values
y_pred4 = combine_labels(pd.Series(y_pred))
y_val_combined = combine_labels(pd.Series(y_val))

# Calculate the accuracy based on the combined labels
accuracy = accuracy_score(y_val_combined, y_pred4)
print(f'Validation Accuracy: {accuracy}')

# Optionally, you can print the classification report as well
print(classification_report(y_val_combined, y_pred4))


Validation Accuracy: 0.88
              precision    recall  f1-score   support

      Aquari       1.00      0.91      0.95        11
       Cybex       0.88      0.78      0.82         9
    Emotivor       0.80      0.89      0.84         9
      Faerix       0.75      1.00      0.86         3
     Florian       0.83      1.00      0.91        10
     Mythron       1.00      0.80      0.89         5
      Nexoon       0.86      0.86      0.86         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.83      0.71      0.77         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.88        75
   macro avg       0.88      0.88      0.88        75
weighted avg       0.89      0.88      0.88        75



## Logistic Regression

In [17]:
lr = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
lr.fit(X_train, y_train)
y_pred1 = lr.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred1)))
print(classification_report(y_val, y_pred1))

Accuracy: 0.92
              precision    recall  f1-score   support

      Aquari       1.00      0.73      0.84        11
       Cybex       1.00      0.89      0.94         9
    Emotivor       0.89      0.89      0.89         9
      Faerix       1.00      1.00      1.00         3
     Florian       0.77      1.00      0.87        10
     Mythron       1.00      1.00      1.00         5
      Nexoon       1.00      1.00      1.00         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.88      1.00      0.93         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.92        75
   macro avg       0.94      0.94      0.93        75
weighted avg       0.93      0.92      0.92        75



## Support Vector Machines (SVM)

In [18]:
svc = LinearSVC()
svc.fit(X_train, y_train)
y_pred2 = svc.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred2)))
print(classification_report(y_val, y_pred2))

Accuracy: 0.84
              precision    recall  f1-score   support

      Aquari       1.00      0.73      0.84        11
       Cybex       1.00      0.89      0.94         9
    Emotivor       0.60      0.67      0.63         9
      Faerix       0.75      1.00      0.86         3
     Florian       0.77      1.00      0.87        10
     Mythron       1.00      0.80      0.89         5
      Nexoon       1.00      1.00      1.00         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.57      0.57      0.57         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.84        75
   macro avg       0.86      0.85      0.85        75
weighted avg       0.86      0.84      0.84        75



/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


## Naive Bayes

In [19]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred3 = mnb.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val,y_pred3)))
print(classification_report(y_val, y_pred3))

Accuracy: 0.6933333333333334
              precision    recall  f1-score   support

      Aquari       1.00      0.82      0.90        11
       Cybex       1.00      0.56      0.71         9
    Emotivor       1.00      0.11      0.20         9
      Faerix       0.00      0.00      0.00         3
     Florian       1.00      0.60      0.75        10
     Mythron       0.45      1.00      0.62         5
      Nexoon       0.64      1.00      0.78         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.43      0.86      0.57         7
     Zorblax       0.58      1.00      0.74         7

    accuracy                           0.69        75
   macro avg       0.71      0.68      0.62        75
weighted avg       0.80      0.69      0.66        75



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Gradient Boost

In [20]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define Gradient Boosting model with valid max_features
gbc = GradientBoostingClassifier(n_estimators=100, max_features='sqrt', max_depth=4, random_state=37, verbose=1)

# Train the model
gbc.fit(X_train, y_train)

# Predict using the trained model
y_pred5 = gbc.predict(X_val)

# Print the accuracy and classification report
print(accuracy_score(y_val, y_pred5))
print(classification_report(y_val, y_pred5))


      Iter       Train Loss   Remaining Time 
         1           2.0686            1.83s
         2           1.8451            1.80s
         3           1.6760            1.72s
         4           1.5424            1.72s
         5           1.4614            1.69s
         6           1.3561            1.70s
         7           1.2759            1.68s
         8           1.1943            1.67s
         9           1.1440            1.66s
        10           1.0751            1.66s
        20           0.6364            1.35s
        30           0.4212            1.14s
        40           0.2915            0.99s
        50           0.2103            0.82s
        60           0.1536            0.69s
        70           0.1147            0.52s
        80           0.0859            0.34s
        90           0.0654            0.17s
       100           0.0496            0.00s
0.76
              precision    recall  f1-score   support

      Aquari       0.83      0.91      

## Adaboost Random Forest

### Hyperparamter Tuning

In [41]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for AdaBoost
param_grid_abc = {
    'base_estimator__n_estimators': [50, 100, 150],
    'base_estimator__max_depth': [5, 9, 15],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150]
}

# Create a GridSearchCV object
grid_search_abc = GridSearchCV(abc, param_grid_abc, cv=5, scoring='accuracy')

# Fit the grid search
grid_search_abc.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search_abc.best_params_)
print("Best Score:", grid_search_abc.best_score_)

# Use the best estimator to predict
best_abc = grid_search_abc.best_estimator_
y_pred_best = best_abc.predict(X_val)
print("Best Accuracy: " + str(accuracy_score(y_val, y_pred_best)))
print(classification_report(y_val, y_pred_best))

from sklearn.model_selection import cross_val_score

# Evaluate using cross-validation
scores = cross_val_score(abc, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", scores)
print("Mean CV Score:", scores.mean())


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Best Parameters: {'base_estimator__max_depth': 15, 'base_estimator__n_estimators': 100, 'learning_rate': 0.05, 'n_estimators': 100}
Best Score: 0.7788235294117647
Best Accuracy: 0.8666666666666667
              precision    recall  f1-score   support

      Aquari       1.00      0.82      0.90        11
       Cybex       0.88      0.78      0.82         9
    Emotivor       0.80      0.89      0.84         9
      Faerix       0.75      1.00      0.86         3
     Florian       0.77      1.00      0.87        10
     Mythron       1.00      0.80      0.89         5
      Nexoon       0.86      0.86      0.86         7
     Quixnar       1.00      0.86      0.92         7
     Sentire       0.83      0.71      0.77         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.87        75
   macro avg       0.88      0.87      0.87        75
weighted avg       0.88      0.87      0.87        75



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Cross-Validation Scores: [0.82352941 0.78823529 0.77647059 0.75294118 0.74117647]
Mean CV Score: 0.7764705882352941


In [42]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=37)
abc= AdaBoostClassifier(base_estimator=rfc, learning_rate=0.05, n_estimators=100)
abc.fit(X_train, y_train)
y_pred7= abc.predict(X_val)
print("Accuracy: "+str(accuracy_score(y_val, y_pred7)))
print(classification_report(y_val, y_pred7))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy: 0.84
              precision    recall  f1-score   support

      Aquari       1.00      0.82      0.90        11
       Cybex       0.88      0.78      0.82         9
    Emotivor       0.80      0.89      0.84         9
      Faerix       0.67      0.67      0.67         3
     Florian       0.77      1.00      0.87        10
     Mythron       0.80      0.80      0.80         5
      Nexoon       0.86      0.86      0.86         7
     Quixnar       0.86      0.86      0.86         7
     Sentire       0.83      0.71      0.77         7
     Zorblax       0.86      0.86      0.86         7

    accuracy                           0.84        75
   macro avg       0.83      0.82      0.82        75
weighted avg       0.85      0.84      0.84        75



## Ensemble Voting Model

In [43]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Function to combine labels
def combine_labels(y):
    return y.replace({
        'Faerix_Group1': 'Faerix',
        'Faerix_Group2': 'Faerix',
        'Emotivor_Group1': 'Emotivor',
        'Emotivor_Group2': 'Emotivor'
    })

# Custom classifier that integrates RandomForestClassifier and label combining
class CustomRandomForestClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=300, max_depth=15, random_state=37, class_weight='balanced'):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.class_weight = class_weight
        self.rfc = RandomForestClassifier(n_estimators=self.n_estimators,
                                          max_depth=self.max_depth,
                                          random_state=self.random_state,
                                          class_weight=self.class_weight)

    def fit(self, X, y):
        self.rfc.fit(X, y)
        return self

    def predict(self, X):
        y_pred = self.rfc.predict(X)
        return combine_labels(pd.Series(y_pred))

    def predict_proba(self, X):
        return self.rfc.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'n_estimators': self.n_estimators,
            'max_depth': self.max_depth,
            'random_state': self.random_state,
            'class_weight': self.class_weight
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        self.rfc = RandomForestClassifier(n_estimators=self.n_estimators,
                                          max_depth=self.max_depth,
                                          random_state=self.random_state,
                                          class_weight=self.class_weight)
        return self

# Instantiate the custom classifier
custom_rfc = CustomRandomForestClassifier(n_estimators=300, max_depth=15, random_state=37, class_weight='balanced')

class CustomLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, C=2, max_iter=1000, n_jobs=-1):
        self.C = C
        self.max_iter = max_iter
        self.n_jobs = n_jobs
        self.lr = LogisticRegression(C=self.C, max_iter=self.max_iter, n_jobs=self.n_jobs)

    def fit(self, X, y):
        self.lr.fit(X, y)
        return self

    def predict(self, X):
        y_pred = self.lr.predict(X)
        return combine_labels(pd.Series(y_pred))

    def predict_proba(self, X):
        return self.lr.predict_proba(X)

    def get_params(self, deep=True):
        return {
            'C': self.C,
            'max_iter': self.max_iter,
            'n_jobs': self.n_jobs
        }

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        self.lr = LogisticRegression(C=self.C, max_iter=self.max_iter, n_jobs=self.n_jobs)
        return self

# Instantiate and use the custom logistic regression classifier
custom_lr = CustomLogisticRegression(C=2, max_iter=1000, n_jobs=-1)

# Define the other classifiers
mnb = MultinomialNB()
rfc = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=37)
lr = LogisticRegression(C=2, max_iter=1000, n_jobs=-1)
svc = SVC(probability=True)
abc= AdaBoostClassifier(base_estimator=rfc, learning_rate=0.05, n_estimators=100)

total_f1 = 93.314794+	84.582965+	61.984111	+87.189813	+80.619854
weights = {
        'Logistic Regression': 102 / total_f1,
        'SVM': 84 / total_f1,
        'Naive Bayes': 69.33 / total_f1,
        'Random Forest': 95 / total_f1,
        'AdaBoost': 92 / total_f1
    }
# Create the VotingClassifier with your custom classifier
ec = VotingClassifier(
    estimators=[
        ('Multinominal NB', mnb),
        ('Custom Random Forest', custom_rfc),
        ('Custom Logistic Regression', custom_lr),
        ('Support Vector Machine', svc),
        ('Adaboost Random Forest', abc),
    ],
    voting='soft',

    weights = [weights['Naive Bayes'], weights['Random Forest'], weights['Logistic Regression'], weights['SVM'], weights['AdaBoost']]
)

# Fit the VotingClassifier
ec.fit(X_train, y_train)

# Predict and evaluate
y_pred6 = ec.predict(X_val)
print(accuracy_score(y_val, y_pred6))
print(classification_report(y_val, y_pred6))


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


0.9333333333333333
              precision    recall  f1-score   support

      Aquari       1.00      0.91      0.95        11
       Cybex       1.00      0.89      0.94         9
    Emotivor       0.89      0.89      0.89         9
      Faerix       1.00      1.00      1.00         3
     Florian       0.91      1.00      0.95        10
     Mythron       1.00      1.00      1.00         5
      Nexoon       1.00      0.86      0.92         7
     Quixnar       0.86      0.86      0.86         7
     Sentire       0.88      1.00      0.93         7
     Zorblax       0.88      1.00      0.93         7

    accuracy                           0.93        75
   macro avg       0.94      0.94      0.94        75
weighted avg       0.94      0.93      0.93        75



In [44]:
Comparison_unibi = pd.DataFrame({'Logistic Regression': [accuracy_score(y_val,y_pred1)*100,f1_score(y_val,y_pred1,average='macro')*100,recall_score(y_val, y_pred1,average='micro')*100,precision_score(y_val, y_pred1,average='micro')*100],
                            'SVM':[accuracy_score(y_val,y_pred2)*100,f1_score(y_val,y_pred2,average='macro')*100,recall_score(y_val, y_pred2,average='micro')*100,precision_score(y_val, y_pred2,average='micro')*100],
                           'Naive Bayes':[accuracy_score(y_val,y_pred3)*100,f1_score(y_val,y_pred3,average='macro')*100,recall_score(y_val, y_pred3,average='micro')*100,precision_score(y_val, y_pred3,average='micro')*100],
                           'Random Forest':[accuracy_score(y_val,y_pred4)*100,f1_score(y_val,y_pred4,average='macro')*100,recall_score(y_val, y_pred4,average='micro')*100,precision_score(y_val, y_pred4,average='micro')*100],
                           'GradientBoosting':[accuracy_score(y_val,y_pred5)*100,f1_score(y_val,y_pred5,average='macro')*100,recall_score(y_val, y_pred5,average='micro')*100,precision_score(y_val, y_pred5,average='micro')*100],
                           'Ensembled':[accuracy_score(y_val,y_pred6)*100,f1_score(y_val,y_pred6,average='macro')*100,recall_score(y_val, y_pred6,average='micro')*100,precision_score(y_val, y_pred6,average='micro')*100],
                           'Adaboost':[accuracy_score(y_val,y_pred7)*100,f1_score(y_val,y_pred7,average='macro')*100,recall_score(y_val, y_pred7,average='micro')*100,precision_score(y_val, y_pred7,average='micro')*100],
})

In [45]:
Comparison_unibi.rename(index={0:'Accuracy',1:'F1_score', 2: 'Recall',3:'Precision'}, inplace=True)
Comparison_unibi.head()

,Logistic Regression,SVM,Naive Bayes,Random Forest,GradientBoosting,Ensembled,Adaboost
Accuracy,92.000000,84.000000,69.333333,88.000000,76.000000,93.333333,84.000000
F1_score,93.314794,84.582965,61.984111,87.559222,74.700038,93.817137,82.425259
Recall,92.000000,84.000000,69.333333,88.000000,76.000000,93.333333,84.000000
Precision,92.000000,84.000000,69.333333,88.000000,76.000000,93.333333,84.000000
